<a href="https://colab.research.google.com/github/millivio/Brain_Tumor_Classification/blob/main/MILLI_FYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting  Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import numpy as np
from matplotlib import pyplot as plt
import h5py
import cv2
from sklearn.utils import shuffle
#from PIL import Image

In [3]:
Xtrain = np.ndarray(shape = (1864,512,512)) #1564 + 500 + 1000
Ytrain = np.zeros(1864)
Xtrain, Ytrain = shuffle(Xtrain, Ytrain)

#Xval = np.ndarray(shape = (500,512,512)) #1564 + 500 + 1000
#Yval = np.zeros(500)

Xtest = np.ndarray(shape = (1200,512,512)) #1564 + 500 + 1000
Ytest = np.zeros(1200)

train_count = 0
test_count= 0
#val_count = 0

In [4]:
i = 1
while i <3065:   
    if train_count < 1864:
        if i < 767:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor1.zip (Unzipped Files)/'
        elif i < 1533:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor3.zip (Unzipped Files)/'
        elif i < 2299:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor2.zip (Unzipped Files)/'
        else:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor4.zip (Unzipped Files)/'

        f = h5py.File(path+str(i)+'.mat','r')
        gr = f.get('cjdata')
        im = np.array(gr.get('image'))
        norm_image = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) #for normalizing the images of dataset
        norm_image = cv2.resize(norm_image, (512,512), interpolation = cv2.INTER_AREA)
        Xtrain[train_count] = np.array(norm_image)
        Ytrain[train_count] = gr.get('label')[()]
        train_count +=1
        i+=1

    if test_count < 1200:
        if i < 767:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor1.zip (Unzipped Files)/'
        elif i < 1533:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor3.zip (Unzipped Files)/'
        elif i < 2299:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor2.zip (Unzipped Files)/'
        else:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor4.zip (Unzipped Files)/'

        f = h5py.File(path+str(i)+'.mat','r')
        gr = f.get('cjdata')
        im = np.array(gr.get('image'))
        norm_image = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) #for normalizing the images of dataset
        norm_image = cv2.resize(norm_image, (512,512), interpolation = cv2.INTER_AREA)

        Xtest[test_count] = np.array(norm_image)
        Ytest[test_count] = gr.get('label')[()]
        test_count +=1
        i+=1
    #print(i)

#    print(np.max(norm_image))
#    plt.imshow(norm_image, cmap="gray") 
#  plt.show()
#    print((gr.get('label')[()])) #cjdata.label: 1 for meningioma, 2 for glioma, 3 for pituitary tumor

In [5]:
#a,b,c = 0,0,0
#for i in range(0,500):
#  if Yval[i] == 1:
#    a+=1
#  elif Yval[i] == 2:
#    b+=1
#  elif Yval[i] == 3:
#    c+=1
#print(a,b,c)

In [6]:
a,b,c = 0,0,0
for i in range(0,1864):
  if Ytrain[i] == 1:
    a+=1
  elif Ytrain[i] == 2:
    b+=1
  elif Ytrain[i] == 3:
    c+=1
print(a,b,c)

355 1044 465


In [7]:
a,b,c = 0,0,0
for i in range(0,1200):
  if Ytest[i] == 1:
    a+=1
  elif Ytest[i] == 2:
    b+=1
  elif Ytest[i] == 3:
    c+=1
print(a,b,c)

353 382 465


Reshaping data

In [8]:
Xtrain = Xtrain.reshape(Xtrain.shape[0],512,512,1)
#Xval = Xval.reshape(Xval.shape[0],512,512,1)
Xtest = Xtest.reshape(Xtest.shape[0],512,512,1)

Ytrain = Ytrain-1
#Yval = Yval - 1
Ytest = Ytest - 1

#print(Yval.shape)

Importing Tensorflow

In [9]:
import tensorflow as tf

In [10]:

# build a sequential model


cnnmodel = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32,(3,3),input_shape=(512,512,1),activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),

      tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.BatchNormalization(),

      tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.BatchNormalization(),

      tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.BatchNormalization(),

     # tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
     # tf.keras.layers.MaxPooling2D((2,2)),
     # tf.keras.layers.BatchNormalization(),

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(1000, activation='relu'),
      #tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Dense(500, activation='relu'),
      
      tf.keras.layers.Dense(100, activation='relu'),
#      tf.keras.layers.Dropout(0.1),

      tf.keras.layers.Dense(3, activation='softmax')

])

In [11]:
#0 for meningioma, 1 for glioma, 2 for pituitary tumor
cn = ['meningioma','glioma', 'pituitary']
cnnmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 510, 510, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 253, 253, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 128)       0

In [12]:
# compile model
cnnmodel.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# fit on data for 30 epochs
cnnmodel.fit(Xtrain, tf.keras.utils.to_categorical(Ytrain),epochs=20)

Epoch 1/20
59/59 [==============================] - 36s 606ms/step - loss: 11.3333 - accuracy: 0.6690
Epoch 2/20
59/59 [==============================] - 35s 598ms/step - loss: 2.5184 - accuracy: 0.7935
Epoch 3/20
59/59 [==============================] - 35s 598ms/step - loss: 0.9490 - accuracy: 0.8267
Epoch 4/20
59/59 [==============================] - 35s 597ms/step - loss: 0.5832 - accuracy: 0.8707
Epoch 5/20
59/59 [==============================] - 35s 597ms/step - loss: 0.3946 - accuracy: 0.8879
Epoch 6/20
59/59 [==============================] - 35s 596ms/step - loss: 0.2912 - accuracy: 0.9067
Epoch 7/20
59/59 [==============================] - 35s 597ms/step - loss: 0.2312 - accuracy: 0.9233
Epoch 8/20
59/59 [==============================] - 35s 598ms/step - loss: 0.3454 - accuracy: 0.9024
Epoch 9/20
59/59 [==============================] - 35s 597ms/step - loss: 0.1857 - accuracy: 0.9431
Epoch 10/20
59/59 [==============================] - 35s 599ms/step - loss: 0.2572 - accur

In [13]:
tf.keras.utils.to_categorical(Ytrain).shape

(1864, 3)

In [14]:
np.unique(Ytrain)

array([0., 1., 2.])

In [15]:
#del Xtrain, Ytrain
y_pred = cnnmodel.predict(Xtest)

In [16]:

prediction = tf.math.argmax(y_pred, axis=1)
equality = tf.math.equal(prediction, Ytest)
accuracy = tf.math.reduce_mean(tf.cast(equality, tf.float32))

In [17]:
print(accuracy)

tf.Tensor(0.95666665, shape=(), dtype=float32)


In [18]:
cn[np.argmax(y_pred[140])]

'meningioma'